# DDM with CNN
Generating training data

## Importing the necessary modules

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import os
import pickle
import yaml
import glob

import numpy as np #numerical python used for working with arrays, mathematical operations
import xarray as xr #package for labeling and adding metadata to multi-dimensional arrays

import sys
#sys.path.append("C:/Users/rmcgorty/Documents/GitHub/PyDDM/PyDDM") #must point to the PyDDM folder
#sys.path.append("C:\\Users\\svc-scst291lab\\Downloads\\PyDDM-main\\PyDDM-main\PyDDM")
#sys.path.append("C:\\Users\\svc-scst255lab\\Documents\\GitHub\\PyDDM\\PyDDM")
sys.path.append("C:\\Users\\jamss\Downloads\\PyDDM-main\\PyDDM-main\\PyDDM")
import ddm_analysis_and_fitting as ddm
#import TCF_Training_Prep

## Creating dictionary of metadata 

In [2]:
ddm_analysis_parameters_str = """
DataDirectory: 'C:\\Users\\jamss\\Downloads\\StevenData\\1umBead_DexGrad_Control_52min_0.0mm_Sweep_1\\'
FileName: '1umBead_DexGrad_Control_52min_0.0mm_Sweep_1_MMStack_Pos0.ome.tif'
Metadata:
  pixel_size: .187 # size of pixel in um
  frame_rate: 25 #frames per second
Analysis_parameters:
  starting_frame_number: 0
  ending_frame_number: 8000
  number_lag_times: 100
  last_lag_time: 600
  crop_to_roi: [0,256,0,256]
  binning: yes
  bin_size: 2
  use_windowing_function: no
Fitting_parameters:
  model: 'DDM Matrix - Single Exponential'
  Tau: [1.0, 0.001, 10]
  StretchingExp: [1.0, 0.5, 3.1]
  Amplitude: [1e2, 1, 1e6]
  Background: [2.5e4, 0, 1e7]
  Good_q_range: [5, 20]
  Auto_update_good_q_range: True
"""
parameters_as_dictionary = yaml.safe_load(ddm_analysis_parameters_str)

## Looping through files and q to find the time dependent DDM matrix

In [3]:
dict_datafile = [k for k in os.listdir(parameters_as_dictionary['DataDirectory']) if 'tif' in k]
print(dict_datafile)

['1umBead_DexGrad_Control_52min_0.0mm_Sweep_1_MMStack_Pos0.ome.tif']


In [4]:
for data_file_name in dict_datafile:
    
    parameters_as_dictionary['FileName'] = data_file_name
    ddm_calc = ddm.DDM_Analysis(parameters_as_dictionary)

    tlags = np.arange(1,101,dtype=int) #We will compute the DDM matrix for all lag times from 1 frame to 998 frames
    ddm_variability = ddm_calc.variationInDDMMatrix(tlags, save_full_ddmmat=False, velocity = [5,0])
    ddm_variability.to_netcdf("%sddm_variability_matrix%s.nc" % (ddm_calc.data_dir, ddm_calc.filename_for_saving_data)) 


Provided metadata: {'pixel_size': 0.187, 'frame_rate': 25}
Image shape: 8000-by-256-by-256
Number of frames to use for analysis: 8000
Maximum lag time (in frames): 600
Number of lag times to compute DDM matrix: 100
New dimensions after cropping: 256-by-256
Applying binning...
Dimensions after binning (8000, 128, 128), the new pixel size 0.374
Will run DDM computation to correct for velocity...


In [ ]:
parameters_as_dictionary['FileName']